In [ ]:
import pandas as pd
from helpFunctions import *
from Headlines_Vectorizer import *
import datetime as dt
import glob
import math
import os
import os.path
import time
from csv import writer
from datetime import date
from itertools import chain, combinations
import transformers
import numpy as np
import pandas as pd
import pytz
import talib
import pickle
from tensorflow import keras
from keras.models import *
from keras.callbacks import (EarlyStopping, ModelCheckpoint, ReduceLROnPlateau,
                             TensorBoard)
from keras.layers.core import Activation, Dense, Dropout
from keras.models import Sequential, load_model
from keras.optimizers import *
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from keras import Input
from keras import Model
from keras import layers
from keras.models import load_model
from keras.layers import *
from sklearn.metrics import accuracy_score
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import *
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from gensim.models import KeyedVectors
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense
from keras.models import Sequential


## 1 - Parameters

In [ ]:
windows = list(range(5,116,10))
windows = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,25,30,35,40,45,50] 
ws_price = 10
ws_news = 20
windows = [5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95,100]
embedding_model = 'word2vec'
runs = 1    
start = '2021-04-09'
valid_start = '2022-01-01' 
fixed_valid = True                                   
test_start = '2022-04-01'   
end = '2023-03-31' 
re_train = False
ticker = "AAPL"
n_past = 10
sentiment_model =  'ProsusAI/finbert'
tr_epochs = 20
batch_size = 32
model_type = "LSTM with 3 Input Layers"
ONE_DAY = dt.timedelta(1)
Classification = False
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'


cols_news =['GeneralNews','MarketsNews','StockNews']
cols_stock = ['Open', 'MiddleBand', 'UpperBand', 'LowerBand', 'Moving Average',
              'Exp. Weighted MA', 'RSI', 'MoneyFlowIndex', 'FI']

#cols_stock = ['Open']

#cols_news =['GeneralNews', 'MarketsNews']
cols_sentiment = ['GeneralNews_Score', 'MarketsNews_Score', 'StockNews_Score', 'GeneralNews_WeightedScore',
              'MarketsNews_WeightedScore', 'StockNews_WeightedScore', 'GeneralNews_DailyScore', 'MarketsNews_DailyScore', 'StockNews_DailyScore',
              'GeneralNews_Weighted_DailyScore', 'MarketsNews_Weighted_DailyScore', 'StockNews_DailyScore']    
                             
features = [['GeneralNews_Score', 'MarketsNews_Score', 'StockNews_Score', 'GeneralNews_WeightedScore',
              'MarketsNews_WeightedScore', 'StockNews_WeightedScore', 'GeneralNews_DailyScore', 'MarketsNews_DailyScore', 'StockNews_DailyScore',
              'GeneralNews_Weighted_DailyScore', 'MarketsNews_Weighted_DailyScore', 'StockNews_DailyScore'], ['GeneralNews_Score', 'MarketsNews_Score', 'StockNews_Score'], ['GeneralNews_WeightedScore',
              'MarketsNews_WeightedScore', 'StockNews_WeightedScore'], ['GeneralNews_DailyScore', 'MarketsNews_DailyScore', 'StockNews_DailyScore'], ['GeneralNews_Weighted_DailyScore', 'MarketsNews_Weighted_DailyScore', 'StockNews_DailyScore']]
inputs = [cols_news, cols_stock, cols_sentiment]

## 2.1 - Pre Process Functions

In [ ]:
#Some Help functions used for preprocessing and initalization of the dataset

def generateOutputFile(name):
    file_path = r'data/results/'                        # File Path for Results
    timestr = time.strftime("%Y%m%d-%H%M%S")
    file_path = file_path + 'results_' + name + '_' + timestr + '.csv'
    print("-" * 60)
    print("Generating File in " + file_path)
    print("-" * 60)
    header_row = ['Ticker','Features','n_past','Re-Train','RMSE','Accuracy','Actual Return abs', 'Actual Return %', 'Training Epochs', 'Batch Size','Start','End','Model Type', 'Train/Test']
    with open(file_path, 'w') as res:
        writer_object = writer(res)
        writer_object.writerow(header_row)
        res.close()
    
    return file_path

def addDim(arr):
    arr_reshaped = arr.reshape(arr.shape[0], arr.shape[1], arr.shape[2], 1)
    return arr_reshaped

def reshapeArray(arr2d):
    # get the length of each list in the 2D array
    list_length = len(arr2d[0][0])
    # create a new 3D numpy array with shape (num_rows, num_cols, list_length)
    new_shape = (arr2d.shape[0], arr2d.shape[1], list_length)
    arr3d = np.empty(new_shape, dtype=object)
    # loop over each element in the 2D array and assign it to the corresponding position in the 3D array
    for i in range(arr2d.shape[0]):
        for j in range(arr2d.shape[1]):
            for k in range(list_length):
                arr3d[i][j][k] = arr2d[j][i][k]

    return arr3d
 
def configIndex(df, ticker, type):
    if type == "headlines" and (ticker is None or ticker =="marketsNews"):
        df['Datetime'] = df['Date'] + " " + df['Time']
        df.drop(columns = "Unnamed: 0", axis =0)
    elif type == "headlines":
        df['Datetime'] = df['Date']
    df['Datetime'] = pd.to_datetime(df['Datetime'])
    df['Datetime'] = df['Datetime'].dt.tz_localize(None)
    df = df.set_index('Datetime')
    return df

def transformHeadlines(df, s_d):
    df = df.reset_index()
    df['Datetime'] = df['Datetime'] - dt.timedelta(hours=6)
    df['Datetime'] = df['Datetime'].apply(lambda row: parseDatetime(row))
    df['Counter'] = 1
    df.loc[df['Positive'] >= df['Negative'], 'Label'] = 1
    df.loc[df['Positive'] < df['Negative'], 'Label'] = -1
    df.loc[df['Label'] == 1, 'PosNews'] = 1
    df.loc[df['Label'] == -1, 'NegNews'] =1
    df.drop_duplicates(subset = "Headline", keep =False, inplace = True)
    df_grouped = df.groupby('Datetime').agg({'Counter': 'sum' , 'Positive':'sum',
         'Negative': 'sum', 'Label': 'sum', 'PosNews': 'sum', 'NegNews':'sum', 
         'Headline': lambda x: ' '.join(x)})
    df_grouped['length'] = df_grouped['Headline'].apply(lambda x: len(x))
    df_grouped = pd.merge(s_d, df_grouped, 'left' , left_index = True, right_index = True)
    df_grouped['Weighted_Score'] = df_grouped['Label'] / df_grouped['Counter']
    df_grouped['Score'] = df_grouped['Positive'] - df_grouped['Negative']
    df_grouped['Date'] = df_grouped.index.date
    df_grouped['Daily_Score'] = df_grouped['Score'].groupby(df_grouped['Date']).cumsum()
    df_grouped['Daily_Weighted_Score'] =df_grouped['Label'].groupby(df_grouped['Date']).cumsum()/ df_grouped['Counter'].groupby(df_grouped['Date']).cumsum() 
    return df_grouped

def splitTrainTest(x_vector, x_prices, x_sentiments, y1, y2, stock_data, split_test, split_valid):
    train_length = len(stock_data[stock_data.index < split_valid]) - n_past
    valid_length = len(stock_data[stock_data.index < split_test]) - n_past
    x_vector_train, x_prices_train, x_sentiments_train, y_train1, y_train2 = x_vector[:train_length], x_prices[:train_length], x_sentiments[:train_length], y1[:train_length], y2[:train_length]
    x_vector_valid, x_prices_valid, x_sentiments_valid, y_valid1, y_valid2 = x_vector[train_length:valid_length], x_prices[train_length: valid_length], x_sentiments[train_length:valid_length],y1[train_length:valid_length],y2[train_length:valid_length]
    valid_length = valid_length +1
    x_vector_test, x_prices_test, x_sentiments_test, y_test1, y_test2 = x_vector[valid_length:], x_prices[valid_length:], x_sentiments[valid_length:], y1[valid_length:], y2[valid_length:]
    return x_vector_train, x_prices_train,x_sentiments_train, y_train1, y_train2, x_vector_test, x_prices_test, x_sentiments_test, y_test1, y_test2, x_vector_valid, x_prices_valid, x_sentiments_valid, y_valid1, y_valid2

def splitTrainTest_noValid(x_vector, x_prices, x_sentiments, y1, y2, stock_data, split_test, split_valid):
    train_length = len(stock_data[stock_data.index < split_test]) - n_past
    x_vector_train, x_prices_train, x_sentiments_train, y_train1, y_train2 = x_vector[:train_length], x_prices[:train_length], x_sentiments[:train_length], y1[:train_length], y2[:train_length]
    x_vector_test, x_prices_test, x_sentiments_test, y_test1, y_test2 = x_vector[train_length+1:], x_prices[train_length+1:], x_sentiments[train_length+1:], y1[train_length+1:], y2[train_length+1:]
    return x_vector_train, x_prices_train,x_sentiments_train, y_train1, y_train2, x_vector_test, x_prices_test, x_sentiments_test, y_test1, y_test2


def createDatasets(df_complete, n_past):
#generate subset for different inputs
        x_price = []
        x_sentiment = []
        y1 = []
        y2 = []

        for i, row in df_complete.iterrows():
            x_price.append(row[cols_stock])
            x_sentiment.append(row[cols_sentiment])
            y1.append(row['Actual Direction'])
            #y2.append(row['Target'])
            y2.append(row['Change'])


        sc = MinMaxScaler()
        sc_predict = MinMaxScaler()

        x_price, x_sentiment, y1,y2 =  np.array(x_price), np.array(x_sentiment), np.array(y1), np.array(y2)
        x_price, x_sentiment, y1,y2 =  x_price.astype(float), x_sentiment.astype(float), y1.astype(float), y2.astype(float)
        x_price = sc.fit_transform(x_price)
        x_sentiment = sc.fit_transform(x_sentiment)
        y2 = y2.reshape(-1, 1)
        y2 = sc_predict.fit_transform(y2)


        #Create dataset incl. lookback
        x_prices = []
        x_sentiments = []
        y1_ = []
        y2_ = []

        for i in range(n_past-1, len(y1)):
            x_prices.append(x_price[i - n_past+1:i+1,: x_price.shape[1]])
            x_sentiments.append(x_sentiment[i-n_past+1:i+1,: x_sentiment.shape[1]])
            y1_.append(y1[i])
            y2_.append(y2[i])

        x_prices, x_sentiments, y1_, y2_= np.array(x_prices), np.array(x_sentiments), np.array(y1_),  np.array(y2_)

        return x_prices, x_sentiments, y1_, y2_, sc, sc_predict


def createlookback(data, lookback):
    samples = data.shape[0] - lookback
    features = data.shape[2]
    lookback_data = np.zeros((samples+1, lookback, features))
    for i in range(samples+1):
        lookback_data[i] = data[i:i+lookback]
    return lookback_data

def lookback(data, lookback_window):
    data_list = []
    for i in range(len(data) - lookback_window+1):
        window = data[i:(i + lookback_window)]
        data_list.append(window)
    return np.array(data_list).astype('float32')

def create_input_arrays(df, ws_price, ws_news):
    input_prices = df[cols_stock]
    sentiment_scores = df[cols_sentiment]
    labels = df['Actual Direction']
    prices = df['Close']
    input_prices_array = []
    sentiment_scores_array = []
    labels_array = []
    prices_array = []
    for i in range(len(df) - ws_price+1):
        input_prices_array.append(input_prices.iloc[i:i+ws_price].values)
        labels_array.append(labels.iloc[i+ws_price-1])
        prices_array.append(prices.iloc[i+ws_price-1])

    for i in range(len(df) - ws_news+1):
        sentiment_scores_array.append(sentiment_scores.iloc[i:i+ws_news].values)


    input_prices_array,sentiment_scores_array, labels_array, prices_array = np.array(input_prices_array), np.array(sentiment_scores_array), np.array(labels_array), np.array(prices_array)

    prices_array =np.reshape(prices_array, (prices_array.shape[0], 1))
    labels_array = np.reshape(labels_array,(labels_array.shape[0],1) )

    sc_predict = MinMaxScaler()
    prices_array = sc_predict.fit_transform(prices_array)

    
    return input_prices_array, sentiment_scores_array, labels_array, prices_array, sc_predict


import torch.nn.functional as F
def flatTensorNumpy(encoded_data_general):
    pad_size = 22
    for i in range(len(encoded_data_general)):
        if not isinstance(encoded_data_general[i], torch.FloatTensor):
            encoded_data_general[i] = torch.zeros((pad_size, 768))

        if len(encoded_data_general[i]) < pad_size:
            encoded_data_general[i] =F.pad(encoded_data_general[i], (0, 0,0,pad_size -len(encoded_data_general[i])), value=0)


    # Concatenate the tensors along a new dimension to flatten them
    flattened_tensor = torch.cat([t.unsqueeze(0) for t in encoded_data_general], dim=0)
    return flattened_tensor.numpy()


## 2.2 - Modelling Functions

In [ ]:
def modelClass_incl_Embedding_directConcat(window_size, embedding_model):
    
    # input layers
    stock_prices_input = Input(shape=(window_size, len(cols_stock)))
    sentiment_scores_input = Input(shape=(window_size, len(cols_sentiment)))
    news_input = Input(shape=(window_size, 768, len(cols_news)))
    
    # reshape news_input
    reshaped_news_input = Reshape((window_size, 768* len(cols_news)))(news_input)

    # concatenate inputs
    merged_input = Concatenate()([stock_prices_input, sentiment_scores_input, reshaped_news_input])

    # LSTM models
    lstm1 = LSTM(128, return_sequences=True)(merged_input)
    lstm2 = LSTM(64, return_sequences=True)(lstm1)

    # merge the output
    merged = LSTM(64)(lstm2)

    # output layer
    direction_output = Dense(1, activation='sigmoid', name='direction_output')(merged)
    price_output = Dense(1, activation='linear', name='price_output')(merged)

    # create the model
    model = Model(inputs=[stock_prices_input, sentiment_scores_input, news_input], outputs=[direction_output, price_output])

    model.compile(optimizer='adam', 
                  loss={'direction_output': 'binary_crossentropy', 'price_output':  'mean_squared_error'},
                  metrics={'direction_output': ['accuracy', 'binary_crossentropy'], 'price_output': ['mean_squared_error']})

    return model

def modelClass_All(window_size, news_amount):
    # input layers
    stock_prices_input = Input(shape=(window_size, len(cols_stock)))
    embedding_input = Input(shape=(window_size, news_amount, 384, len(cols_news)))
    sentiment_scores_input = Input(shape=(window_size,  len(cols_sentiment)))

    filters = 8
    

    if news_amount == 1:
        conv_layer = Conv2D(filters=filters, kernel_size=(news_amount, 384), activation='relu')(embedding_input)
        reshape_layer = Reshape((window_size, 1, filters))(conv_layer)
        pooling_layer = MaxPooling2D(pool_size=(1,1))(reshape_layer)
        reshape_layer = Reshape((window_size, news_amount * filters))(conv_layer)
    else:
        conv_layer = Conv2D(filters=filters, kernel_size=(3, 384), activation='relu')(embedding_input)
        reshape_layer = Reshape((window_size, filters, filters))(conv_layer)
        pooling_layer = MaxPooling2D(pool_size=(1, 2))(reshape_layer)
        reshape_layer = Reshape((window_size, filters * int(filters/2)))(pooling_layer)

    #reshape_layer = Reshape((window_size, filters * int(filters/2)))(pooling_layer)


    # LSTM models
    stock_prices_lstm = LSTM(128, return_sequences= True)(stock_prices_input)
    stock_prices_lstm = LSTM(64, return_sequences= True)(stock_prices_lstm)
    stock_prices_lstm = LSTM(32, return_sequences= False)(stock_prices_lstm)
    stock_prices_lstm = Dropout(0.25)(stock_prices_lstm)

    sentiment_scores_lstm = LSTM(128, return_sequences= True)(sentiment_scores_input)
    sentiment_scores_lstm = LSTM(64, return_sequences= True)(sentiment_scores_lstm)
    sentiment_scores_lstm = LSTM(32, return_sequences= False)(sentiment_scores_lstm)
    sentiment_scores_lstm = Dropout(0.25)(sentiment_scores_lstm)

    embedding_lstm = LSTM(256, return_sequences= True)(reshape_layer)
    embedding_lstm = LSTM(128, return_sequences= True)(embedding_lstm)
    embedding_lstm = LSTM(64, return_sequences= False)(embedding_lstm)
    embedding_lstm = Dropout(0.25)(embedding_lstm)

    # merge the output
    merged = Concatenate()([embedding_lstm, sentiment_scores_lstm, stock_prices_lstm])
    #merged = Dropout(0.25)(merged)

    # output layer
    direction_output = Dense(1, activation='sigmoid', name='direction_output')(merged)
    price_output = Dense(1, activation='linear', name='price_output')(merged)


    # create the model
    model = Model(inputs=[embedding_input, sentiment_scores_input, stock_prices_input], outputs= [direction_output, price_output])
    #model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])
    #model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model.compile(optimizer='adam', 
             loss={'direction_output': 'binary_crossentropy', 'price_output': ['mean_squared_error']},
              #loss_weights={'direction_output': 0.8, 'price_output': 0.2},
            metrics={'direction_output': ['accuracy', 'binary_crossentropy'], 'price_output': ['mape']})

    return model

def modelClass_incl_Embedding(window_size, embedding_model):

    # input layers
    stock_prices_input = Input(shape=(window_size, len(cols_stock)))
    sentiment_scores_input = Input(shape=(window_size, len(cols_sentiment)))
    news_input = Input(shape=(window_size, 768,3 ))
    
    # LSTM models
    stock_prices_lstm = LSTM(128, return_sequences=True)(stock_prices_input)
    stock_prices_lstm = LSTM(64, return_sequences=True)(stock_prices_lstm)
    #stock_prices_lstm = Dropout(0.25)(stock_prices_lstm)
    #p_output_stock = Dense(1, activation = 'linear')(stock_prices_lstm)
    #d_output_stock = Dense(1, activation= 'sigmoid')(stock_prices_lstm)

    sentiment_scores_lstm = LSTM(128, return_sequences=True)(sentiment_scores_input)
    sentiment_scores_lstm = LSTM(64, return_sequences= True)(sentiment_scores_lstm)
    #sentiment_scores_lstm = Dropout(0.25)(sentiment_scores_lstm)
    #p_output_sent = Dense(1, activation = 'linear')(sentiment_scores_lstm)
    #d_output_sent = Dense(1, activation='sigmoid')(sentiment_scores_lstm)

    conv = Conv1D(filters=8, kernel_size=3, padding='same', activation='relu', data_format='channels_last')(news_input)
    flat = Flatten()(conv)
    reshaped = Reshape((-1, 128))(flat)
    news_lstm = LSTM(128, return_sequences=True)(reshaped)
    news_lstm = LSTM(64, return_sequences=True)(news_lstm)
    #news_lstm = Dropout(0.25)(news_lstm)
    #p_output_news = Dense(1, activation='linear')(news_lstm)
    #d_output_news = Dense(1, activation='sigmoid')(news_lstm)


    # merge the output
    merged = Concatenate()([news_lstm, sentiment_scores_lstm,stock_prices_lstm])
    merged = LSTM(64) (merged)

    #merge the output

    #price_output = layers.concatenate([p_output_stock, p_output_sent, p_output_news])
    #direction_output = layers.concatenate([d_output_stock, d_output_sent, d_output_news])

    #price_output = Dense(1, activation = 'linear',  name='price_output') (price_output)
    #direction_output = Dense(1, activation = 'sigmoid', name='direction_output')(direction_output)

    # output layer
    direction_output = Dense(1, activation='sigmoid', name='direction_output')(merged)
    price_output = Dense(1, activation='linear', name='price_output')(merged)

    
    # create the model
    model = Model(inputs=[news_input, sentiment_scores_input, stock_prices_input], outputs=[direction_output, price_output])

    #model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model.compile(optimizer='adam', 
            loss={'direction_output': 'binary_crossentropy', 'price_output':  'mean_squared_error'},
            loss_weights={'direction_output': 0.8, 'price_output': 0.2},
            metrics={'direction_output': ['accuracy', 'binary_crossentropy'], 'price_output': ['mean_squared_error']})



    return model

def modelClass_no_Embedding(window_size):
    # input layers
    stock_prices_input = Input(shape=(window_size, len(cols_stock)))
    sentiment_scores_input = Input(shape=(window_size,  len(cols_sentiment)))
    
    # LSTM model
    stock_prices_lstm = LSTM(128, return_sequences= True)(stock_prices_input)
    stock_prices_lstm = LSTM(64, return_sequences= True)(stock_prices_lstm)
    stock_prices_lstm = LSTM(32, return_sequences= False)(stock_prices_lstm)
    stock_prices_lstm = Dropout(0.2)(stock_prices_lstm)
    sentiment_scores_lstm = LSTM(128, return_sequences= True)(sentiment_scores_input)
    sentiment_scores_lstm = LSTM(64, return_sequences= True)(sentiment_scores_lstm)
    sentiment_scores_lstm = LSTM(32, return_sequences= False)(sentiment_scores_lstm)
    sentiment_scores_lstm = Dropout(0.2)(sentiment_scores_lstm)

    # merge the output
    merged = Concatenate()([sentiment_scores_lstm, stock_prices_lstm])
    #merged = Dropout(0.5) (merged)

    # output layer
    direction_output = Dense(1, activation='sigmoid', name='direction_output')(merged)
    price_output = Dense(1, activation='linear', name='price_output')(merged)


    # create the model
    model = Model(inputs=[sentiment_scores_input, stock_prices_input], outputs= [direction_output, price_output])
    model.compile(optimizer='adam', 
              loss={'direction_output': 'binary_crossentropy', 'price_output': ['mean_squared_error']},
              #loss_weights={'direction_output': 0.8, 'price_output': 0.2},
              metrics={'direction_output': ['accuracy', 'binary_crossentropy'], 'price_output': ['mape']})

    return model

def modelClass_no_Embedding_directConcat(window_size):
    # input layers
    stock_prices_input = Input(shape=(window_size, len(cols_stock)))
    sentiment_scores_input = Input(shape=(window_size, len(cols_sentiment)))
    
    # input layers
    combined_input = Concatenate(axis = 2)([stock_prices_input, sentiment_scores_input])

    # LSTM model
    lstm = LSTM(128, return_sequences=True)(combined_input)
    #lstm = Dropout(0.2)(lstm)
    lstm = LSTM(64, return_sequences = True)(lstm)
    #lstm = Dropout(0.2)(lstm)
    lstm = LSTM(32, return_sequences = False)(lstm)
    lstm = Dropout(0.25)(lstm)

    # output layers
    direction_output = Dense(1, activation='sigmoid', name='direction_output')(lstm)
    price_output = Dense(1, activation='linear', name='price_output')(lstm)

    # create the model
    model = Model(inputs=[sentiment_scores_input,stock_prices_input], outputs= [direction_output, price_output])
    model.compile(optimizer='adam', 
              loss={
               'direction_output': 'binary_crossentropy',
                 'price_output': 'mape'
               },
              #loss_weights={'direction_output': 0.8, 'price_output': 0.2},
              metrics={
        'direction_output': ['accuracy', 'binary_crossentropy'], 
        'price_output': 'mape'
        })

    return model

def modelClass_onlySent(window_size):
    # input layers
    sentiment_scores_input = Input(shape=(window_size,  len(cols_sentiment)))
    
    # LSTM model
    sentiment_scores_lstm = LSTM(128, return_sequences= True)(sentiment_scores_input)
    sentiment_scores_lstm = LSTM(64, return_sequences= True)(sentiment_scores_lstm)
    sentiment_scores_lstm = LSTM(32, return_sequences= False)(sentiment_scores_lstm)
    sentiment_scores_lstm = Dropout(0.2)(sentiment_scores_lstm)



    # output layer
    direction_output = Dense(1, activation='sigmoid', name='direction_output')(sentiment_scores_lstm) 
    price_output = Dense(1, activation='linear', name='price_output')(sentiment_scores_lstm )


    # create the model
    model = Model(inputs=[sentiment_scores_input], outputs= [direction_output, price_output])
    model.compile(optimizer='adam', 
              loss={'direction_output': 'binary_crossentropy', 'price_output': ['mean_squared_error']},
              #loss_weights={'direction_output': 0.8, 'price_output': 0.2},
              metrics={'direction_output': ['accuracy', 'binary_crossentropy'], 'price_output': ['mape']})

    return model

def modelClass_onlyEmbedd(window_size):
    # input layers
    embedding_input = Input(shape=(window_size, news_amount, 384, len(cols_news)))

    filters = 8
    if news_amount == 1:
        conv_layer = Conv2D(filters=filters, kernel_size=(news_amount, 384), activation='relu')(embedding_input)
        reshape_layer = Reshape((window_size, 1, filters))(conv_layer)
        pooling_layer = MaxPooling2D(pool_size=(1,1))(reshape_layer)
        reshape_layer = Reshape((window_size, news_amount * filters))(conv_layer)
    else:
        conv_layer = Conv2D(filters=filters, kernel_size=(3, 384), activation='relu')(embedding_input)
        reshape_layer = Reshape((window_size, filters, filters))(conv_layer)
        pooling_layer = MaxPooling2D(pool_size=(1, 2))(reshape_layer)
        reshape_layer = Reshape((window_size, filters * int(filters/2)))(pooling_layer)


    embedding_lstm = LSTM(256, return_sequences= True)(reshape_layer)
    embedding_lstm = LSTM(128, return_sequences= True)(embedding_lstm)
    embedding_lstm = LSTM(64, return_sequences= False)(embedding_lstm)
    embedding_lstm = Dropout(0.5)(embedding_lstm)



    # output layer)
    direction_output = Dense(1, activation='sigmoid', name='direction_output')(embedding_lstm)
    price_output = Dense(1, activation='linear', name='price_output')(embedding_lstm)


    # create the model
    model = Model(inputs=[embedding_input], outputs= [direction_output, price_output])
    model.compile(optimizer='adam', 
             loss={'direction_output': 'binary_crossentropy', 'price_output': ['mean_squared_error']},
              #loss_weights={'direction_output': 0.8, 'price_output': 0.2},
            metrics={'direction_output': ['accuracy', 'binary_crossentropy'], 'price_output': ['mape']})

    return model

def modelClass_only_stock(window_size):
    # input layer
    stock_prices_input = Input(shape=(window_size, len(cols_stock)), name= 'Stock Price Input')

    # LSTM model
    stock_prices_lstm = LSTM(128, return_sequences= True, name = 'LSTM_1')(stock_prices_input)
    #stock_prices_lstm = Dropout(0.25)(stock_prices_lstm)
    stock_prices_lstm = LSTM(64, return_sequences = True, name = 'LSTM_2')(stock_prices_lstm)
    #stock_prices_lstm = Dropout(0.25)(stock_prices_lstm)
    stock_prices_lstm = LSTM(32, name = 'LSTM_3')(stock_prices_lstm)
    stock_prices_lstm = Dropout(0.25, name = 'Dropout')(stock_prices_lstm)
    # output layer
    direction_output = Dense(1, activation='sigmoid', name='direction_output')(stock_prices_lstm)
    price_output = Dense(1, activation='linear', name='price_output')(stock_prices_lstm)



    # create the model
    model = Model(inputs=stock_prices_input, outputs=  [direction_output, price_output])
    #model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model.compile(optimizer='adam', 
              loss={
                'direction_output': 'binary_crossentropy',
                 'price_output': 'mape'},
              #loss_weights={'direction_output': 0.8, 'price_output': 0.2},
              metrics={
        'direction_output': ['accuracy', 'binary_crossentropy'], 
        'price_output': 'mape'})

    return model

def modelClass_Embedd_Price(window_size, news_amount):
    # input layers
    stock_prices_input = Input(shape=(window_size, len(cols_stock)))
    embedding_input = Input(shape=(window_size, news_amount, 384, len(cols_news)))

    filters = 8
    

    if news_amount == 1:
        conv_layer = Conv2D(filters=filters, kernel_size=(news_amount, 384), activation='relu')(embedding_input)
        reshape_layer = Reshape((window_size, 1, filters))(conv_layer)
        pooling_layer = MaxPooling2D(pool_size=(1,1))(reshape_layer)
        reshape_layer = Reshape((window_size, news_amount * filters))(conv_layer)
    else:
        conv_layer = Conv2D(filters=filters, kernel_size=(3, 384), activation='relu')(embedding_input)
        reshape_layer = Reshape((window_size, filters, filters))(conv_layer)
        pooling_layer = MaxPooling2D(pool_size=(1, 2))(reshape_layer)
        reshape_layer = Reshape((window_size, filters * int(filters/2)))(pooling_layer)

    #reshape_layer = Reshape((window_size, filters * int(filters/2)))(pooling_layer)


    # LSTM models
    stock_prices_lstm = LSTM(128, return_sequences= True)(stock_prices_input)
    #stock_prices_lstm = Dropout(0.5)(stock_prices_lstm)
    stock_prices_lstm = LSTM(64, return_sequences= True)(stock_prices_lstm)
    #stock_prices_lstm = Dropout(0.5)(stock_prices_lstm)
    stock_prices_lstm = LSTM(32, return_sequences= False)(stock_prices_lstm)
    stock_prices_lstm = Dropout(0.5)(stock_prices_lstm)
    embedding_lstm = LSTM(256, return_sequences= True)(reshape_layer)
    #embedding_lstm = Dropout(0.5)(embedding_lstm)
    embedding_lstm = LSTM(128, return_sequences= True)(embedding_lstm)
    #embedding_lstm = Dropout(0.5)(embedding_lstm)
    embedding_lstm = LSTM(64, return_sequences= False)(embedding_lstm)
    embedding_lstm = Dropout(0.5)(embedding_lstm)

    # merge the output
    merged = Concatenate()([embedding_lstm, stock_prices_lstm])
    #merged = Dropout(0.25)(merged)

    # output layer)
    direction_output = Dense(1, activation='sigmoid', name='direction_output')(merged)
    price_output = Dense(1, activation='linear', name='price_output')(merged)


    # create the model
    model = Model(inputs=[stock_prices_input, embedding_input], outputs= [direction_output, price_output])
    #model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])
    #model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model.compile(optimizer='adam', 
             loss={'direction_output': 'binary_crossentropy', 'price_output': ['mape']},
              #loss_weights={'direction_output': 0.8, 'price_output': 0.2},
            metrics={'direction_output': ['accuracy', 'binary_crossentropy'], 'price_output': ['mape']})

    return model

def modelAll(window_size, news_amount):
    # input layers
    price_input = Input(shape=(window_size, len(cols_stock)), name='price_input')
    sentiment_input = Input(shape=(window_size, len(cols_sentiment)), name='sentiment_input')
    news_input = Input(shape=(window_size, news_amount, 768, len(cols_news)), name='news_input')

    # embedding layer for news input
    news_embedding = TimeDistributed(Conv1D(filters=32, kernel_size=3, activation='relu'))(news_input)
    news_embedding = TimeDistributed(GlobalMaxPooling1D())(news_embedding)

    # convolutional layer for sentiment input
    sentiment_conv = Conv1D(filters=8, kernel_size=3, activation='relu')(sentiment_input)
    sentiment_pool = GlobalMaxPooling1D()(sentiment_conv)

    # concatenating all inputs
    concat = Concatenate(axis=2)([price_input, sentiment_pool, news_embedding])

    # LSTM layer
    lstm = LSTM(64, activation='relu', dropout=0.2, recurrent_dropout=0.2)(concat)

    # output layers
    price_output = Dense(1, name='price_output')(lstm)

    # compile model
    model = Model(inputs=[price_input, sentiment_input, news_input], outputs=price_output)
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])

    return model

def evalModel(preds, data, evalType, pred_scaler):
    result = data.copy()
    result.loc[result['Close'] > result['Open'], 'Actual Direction'] = 1
    result.loc[result['Close'] <= result['Open'], 'Actual Direction'] = 0
    result['Actual_Change'] = result['Close'] - result['Open']
    if not Classification:
        result['Change'] = pred_scaler.inverse_transform(preds) 
        result.loc[result['Change'] > 0, 'Decision']  = 1
        result.loc[result['Change'] <= 0, 'Decision']  = 0
        result['Prediction'] = result['Change'] + result['Open']
        #MSE = np.square(np.subtract(result['Prediction'], result['Close'])).mean() 
        MSE = np.square(np.subtract(result['Change'], result['Actual_Change'])).mean() 
        RMSE = math.sqrt(MSE)
        #result.loc[result['Prediction'] > result['Open'], 'Decision'] = 1
        #result.loc[result['Prediction'] <= result['Open'], 'Decision'] = 0
        
    else:
        if len(preds[0]) > 1:
            result['Prediction']= preds[0]
            result['Change'] = pred_scaler.inverse_transform(preds[1])
            result['Price_Prediction'] = result['Change'] + result['Open']
            #result['Price_Prediction'] = preds[1]
            MSE = np.square(np.subtract(result['Change'], result['Actual_Change'])).mean() 
            RMSE = math.sqrt(MSE)
        else:
            result['Prediction'] = preds
            RMSE = 0

        result.loc[result['Prediction'] > 0.5, 'Decision']  = 1
        result.loc[result['Prediction'] <= 0.5, 'Decision']  = 0
        



    start_credit = 10000
    credit = start_credit
    # Return Calculation based on actual trades
    for j in result.index:
        amount = credit / result.loc[j, 'Open']              #replace credit with start_credit to ignore Zinseszins
        #amount = 10                    
        #if abs(result.loc[j, 'predicted'] - result.loc[j, 'open']) > 0.5: #Only trade "Big Changes"
        if result.loc[j, 'Decision']:
            result.loc[j, 'return'] = result.loc[j, 'Close'] * amount - result.loc[j, 'Open'] * amount
        else:
            result.loc[j, 'return'] = result.loc[j, 'Open'] * amount - result.loc[j, 'Close'] * amount
        #else:
         #   result.loc[j, 'return'] = 0
        credit = credit + result.loc[j, 'return']
        

    return_abs = credit - start_credit
    return_per = ((credit / start_credit) -1) *100

    if len(preds[0]) > 1:
        print('-' * 60)
        print("Root Mean Square Error:")
        print(RMSE)
        print('-' * 60)
    print("\nAccuracy: ")
    print(accuracy_score(result['Actual Direction'], result['Decision']))
    print(confusion_matrix(result['Actual Direction'], result['Decision']))
    print('-' * 60)
    print("\nActual Return Absolut:")
    print(return_abs)
    print('-' * 60)
    print('-' * 60)
    print("\nActual Return %:")
    print(return_per)
    print('-' * 60)

    timestr = time.strftime("%Y%m%d-%H%M%S")
    result.to_csv('data/results/' + evalType +  "/Results_Evaluation_" + emType + '_' +  str(n_past) + '_' + timestr +'.csv')
    return RMSE, accuracy_score(result['Actual Direction'], result['Decision']), return_abs, return_per


## 3 - Execute PreProcessing

In [ ]:
import torch
#Read in Data
h_general = configIndex(getLatestData("sentiments", None, sentiment_model), None, "sentiments")
h_market = configIndex(getLatestData("sentiments", "marketsNews",sentiment_model), "marketsNews", "sentiments")
h_stock = configIndex(getLatestData("sentiments", ticker, sentiment_model), ticker, "sentiments")
stock_data = configIndex(getLatestData("stock_data", ticker, sentiment_model), ticker, "stock_data")



#create Technical Stock Indicators
close = stock_data['Adj Close']
stock_data['Moving Average'] = talib.SMA(close)
stock_data['Exp Moving Average'] = talib.EMA(close)
stock_data['MACD'] = talib.MACD(close )[1] #0 = Series, 1 = Signal, 2 = Divergence
upper, middle, lower = talib.BBANDS(close)
stock_data['UpperBBA'] = upper
stock_data['MiddleBBA'] = middle
stock_data['LowerBBA'] = lower
stock_data['Momentum'] = talib.MOM(close)
stock_data['Williams_R'] = talib.WILLR(stock_data['High'],stock_data['Low'], close,14)
stock_data['RSI'] = talib.RSI(close)
stock_data['STOCH_K'] = talib.STOCH(stock_data['High'],stock_data['Low'], close)[0]
stock_data['STOCH_D'] = talib.STOCH(stock_data['High'],stock_data['Low'],close)[1]
stock_data['ROCP'] = talib.ROCP(close)



#Transform the Data
h_gen = transformHeadlines(h_general, stock_data)
h_mar = transformHeadlines(h_market, stock_data)
h_stock = transformHeadlines(h_stock, stock_data)


In [ ]:
#Read in encoded Sentence Vectors

with open('0_GeneralNews_Last1_encoded.pkl', 'rb') as f:
    encoded_data_general = pickle.load(f)
    encoded_data_general = reshapeArray(encoded_data_general)


with open('1_MarketNews_Last1_encoded.pkl', 'rb') as f:
    encoded_data_market = pickle.load(f)
    encoded_data_market = reshapeArray(encoded_data_market)


with open('2_StockNews_Last1_encoded.pkl', 'rb') as f:
    encoded_data_stock = pickle.load(f)
    encoded_data_stock = reshapeArray(encoded_data_stock)


with open('0_GeneralNews_Last10_encoded.pkl', 'rb') as f:
    encoded_data_general10 = pickle.load(f)
    encoded_data_general10 = reshapeArray(encoded_data_general10)


with open('1_MarketNews_Last10_encoded.pkl', 'rb') as f:
    encoded_data_market10 = pickle.load(f)
    encoded_data_market10 = reshapeArray(encoded_data_market10)


with open('2_StockNews_Last10_encoded.pkl', 'rb') as f:
    encoded_data_stock10 = pickle.load(f)
    encoded_data_stock10 = reshapeArray(encoded_data_stock10)


with open('0_GeneralNews_Mean_encoded.pkl', 'rb') as f:
    encoded_data_general_mean = pickle.load(f)
    encoded_data_general_mean = reshapeArray(encoded_data_general_mean)


with open('1_MarketNews_Mean_encoded.pkl', 'rb') as f:
    encoded_data_market_mean = pickle.load(f)
    encoded_data_market_mean = reshapeArray(encoded_data_market_mean)


with open('2_StockNews_Mean_encoded.pkl', 'rb') as f:
    encoded_data_stock_mean = pickle.load(f)
    encoded_data_stock_mean = reshapeArray(encoded_data_stock_mean)




In [ ]:
#Construct complete Dataframe
d = {'GeneralNews': h_gen['Headline'], 'MarketsNews': h_mar['Headline'], 'StockNews' :h_stock['Headline'],
        'GeneralNews_Score': h_gen['Score'], 'MarketsNews_Score': h_mar['Score'], 'StockNews_Score': h_stock['Score'],
        'GeneralNews_WeightedScore': h_gen['Weighted_Score'], 'MarketsNews_WeightedScore': h_mar['Weighted_Score'], 'StockNews_WeightedScore': h_stock['Weighted_Score'],
        'GeneralNews_DailyScore': h_gen['Daily_Score'], 'MarketsNews_DailyScore': h_mar['Daily_Score'], 'StockNews_DailyScore': h_stock['Daily_Score'],
        'GeneralNews_Weighted_DailyScore': h_gen['Daily_Weighted_Score'], 'MarketsNews_Weighted_DailyScore': h_mar['Daily_Weighted_Score'], 'StockNews_Weighted_DailyScore': h_stock['Daily_Weighted_Score'],
        'GeneralNewsCounter': h_gen['Counter'], 'MarketsNewsCounter': h_mar['Counter'], 'StockNewsCounter': h_stock['Counter'],
        'GeneralNewsPosCounter': h_gen['PosNews'], 'MarketsNewsPosCounter': h_mar['PosNews'], 'StockNewsPosCounter': h_stock['PosNews'],
        'GeneralNewsNegCounter': h_gen['NegNews'], 'MarketsNewsNegCounter': h_mar['NegNews'], 'StockNewsNegCounter': h_stock['NegNews'],
        }
h_all = pd.DataFrame(data = d)

news_col =['GeneralNews','MarketsNews']

#fill NaN for Text Data with "None", for numerical with 0
for col in news_col:
        h_all[col] = h_all[col].fillna("None")
h_all.fillna(0, inplace = True)
df_complete = pd.merge(h_all, stock_data, "inner", left_index = True, right_index = True)
df_complete.loc[df_complete['Close'] > df_complete['Open'], 'Actual Direction'] = 1
df_complete.loc[df_complete['Close'] <= df_complete['Open'], 'Actual Direction'] = 0
df_complete['Target'] = df_complete['Close']
df_complete['Current_Open'] = df_complete['Open']
df_complete['Change'] = df_complete['Close'] - df_complete['Open']
df_complete['Actual_Change'] = df_complete['Change']
df_complete['Hour'] = df_complete.index.hour


## 4.1 Parmeter Optimization

In [ ]:

cols_stock = [
              'Open', 'High', 'Low', 'Close', 'Adj Close', 
              'Volume', 'Moving Average',
              'Exp Moving Average', 'MACD', 'UpperBBA', 'MiddleBBA', 'LowerBBA',
              'Momentum', 'RSI', 'STOCH_K', 'STOCH_D', 'Williams_R', 'ROCP'
       ]

#Shift all due to no information leak by 1
df_complete[cols_stock] = df_complete[cols_stock].shift(1)

#Add current open price (no leak)
cols_stock.extend(['Current_Open'])

df_complete = df_complete[(df_complete.index > start) & (df_complete.index < end)]



cols_news =['GeneralNews', 'MarketsNews', 'StockNews']
cols_sentiment = ['GeneralNews_Score', 'MarketsNews_Score', 'StockNews_Score', 'GeneralNews_WeightedScore',
              'MarketsNews_WeightedScore', 'StockNews_WeightedScore', 'GeneralNews_DailyScore', 'MarketsNews_DailyScore', 'StockNews_DailyScore',
              'GeneralNews_Weighted_DailyScore', 'MarketsNews_Weighted_DailyScore', 'StockNews_DailyScore']  

#cols_sentiment = ['GeneralNews_WeightedScore','MarketsNews_WeightedScore', 'StockNews_WeightedScore']  

                            
features = [['GeneralNews_Score', 'MarketsNews_Score', 'StockNews_Score', 'GeneralNews_WeightedScore',
              'MarketsNews_WeightedScore', 'StockNews_WeightedScore', 'GeneralNews_DailyScore', 'MarketsNews_DailyScore', 'StockNews_DailyScore',
              'GeneralNews_Weighted_DailyScore', 'MarketsNews_Weighted_DailyScore', 'StockNews_DailyScore'], 
              ['GeneralNews_Score', 'MarketsNews_Score', 'StockNews_Score'], 
              ['GeneralNews_WeightedScore', 'MarketsNews_WeightedScore', 'StockNews_WeightedScore'], 
              ['GeneralNews_DailyScore', 'MarketsNews_DailyScore', 'StockNews_DailyScore'], 
              ['GeneralNews_Weighted_DailyScore', 'MarketsNews_Weighted_DailyScore', 'StockNews_DailyScore'], 
              ['GeneralNews_Score', 'MarketsNews_Score', 'StockNews_Score', 'GeneralNews_WeightedScore','MarketsNews_WeightedScore', 'StockNews_WeightedScore'],
               ['GeneralNews_DailyScore', 'MarketsNews_DailyScore', 'StockNews_DailyScore',
              'GeneralNews_Weighted_DailyScore', 'MarketsNews_Weighted_DailyScore', 'StockNews_DailyScore'],
              ['GeneralNews_Score', 'MarketsNews_Score', 'StockNews_Score', 'GeneralNews_DailyScore', 'MarketsNews_DailyScore', 'StockNews_DailyScore'],
              ['GeneralNews_WeightedScore','MarketsNews_WeightedScore', 'StockNews_WeightedScore','GeneralNews_Weighted_DailyScore', 'MarketsNews_Weighted_DailyScore', 'StockNews_DailyScore']
              ]

features = [['GeneralNews_Score', 'MarketsNews_Score', 'StockNews_Score', 'GeneralNews_WeightedScore',
              'MarketsNews_WeightedScore', 'StockNews_WeightedScore', 'GeneralNews_DailyScore', 'MarketsNews_DailyScore', 'StockNews_DailyScore',
              'GeneralNews_Weighted_DailyScore', 'MarketsNews_Weighted_DailyScore', 'StockNews_DailyScore'],
              ['GeneralNews_Score', 'GeneralNews_WeightedScore', 'GeneralNews_DailyScore', 'GeneralNews_Weighted_DailyScore'],
              ['MarketsNews_Score', 'MarketsNews_WeightedScore', 'MarketsNews_DailyScore', 'MarketsNews_Weighted_DailyScore'],
              ['StockNews_Score', 'StockNews_WeightedScore', 'StockNews_DailyScore', 'StockNews_Weighted_DailyScore']
              ] 

features = [[ 'GeneralNews_DailyScore', 'MarketsNews_DailyScore', 'StockNews_DailyScore'],
              ['GeneralNews_DailyScore'],
              [ 'MarketsNews_DailyScore'],
              [ 'StockNews_DailyScore']
              ] 

features = ['All', 'General', 'Market', 'Stock']
inputs = [cols_news, cols_stock, cols_sentiment]

In [ ]:
# Amount of Experiments
runs = 5     

#Window Size
windows = list(range(5,116,10))
windows = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,25,30,35,40,45,50] 
ws_price = 10
ws_news = 20
windows = [5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95,100]
windows = list(range(1,51,1))
n_past = 41


#Training Set Up
tr_epochs = 50
eps = [50]
batch_size = 32
#batches = [2,8,16,32,64]
re_train = False
model_type = "LSTM with 3 Input Layers"
emType = 'L1'
Classification = True
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

## 4.2 - Experiments and Modelling

In [ ]:
test_start_org = test_start
valid_start_org = valid_start
#Callbacks:
es = EarlyStopping(monitor='val_direction_output_accuracy', min_delta=1e-10, patience=50, verbose=1)
rlr = ReduceLROnPlateau(monitor='val_direction_output_accuracy', factor=0.5, patience=10, verbose=1)
mcp = ModelCheckpoint(filepath='weights.h5', monitor='val_direction_output_accuracy', verbose=1, save_best_only=True, save_weights_only = False)
tb = TensorBoard()



#runs = len(windows)
runs = 1
for run in range(0,runs):
    #n_past = windows[run]
    #tr_epochs = eps[run]
    #for i, window in enumerate(windows):
    for i,feature in enumerate(features):
        test_start = test_start_org
        valid_start= valid_start_org
        print("Start " + start)
        print("Valid Start " + valid_start)
        print("Test Start " + test_start)
        #n_past = window
        
        #cols.extend(list(feature))
        #cols_sentiment = feature
        print(" " * 60)
        #print("Run: " + str(i+1) + "/" +  str(len(windows)) + " with Window = " + str(window))
        print("Run: " + str(i+1) + "/" +  str(len(features)) + " with Features = " + str(cols_sentiment))
        print("-" * 60)

        x_prices, x_sentiments, y1, y2, sc, sc_predict = createDatasets(df_complete,n_past)

        #x_vector = np.stack((encoded_data_general, encoded_data_market, encoded_data_stock),axis = 3)
        news_amount = 10
        if i <= 0:
            cols_news =['GeneralNews', 'MarketsNews', 'StockNews']
            x_vector = np.stack((encoded_data_general10, encoded_data_market10, encoded_data_stock10),axis = 3)
            #x_vector = np.stack((encoded_data_general, encoded_data_market, encoded_data_stock),axis = 3)
            #x_vector = np.stack((encoded_data_general_mean, encoded_data_market_mean, encoded_data_stock_mean),axis = 3)
        elif i == 1:
            cols_news =['GeneralNews']
            x_vector = addDim(encoded_data_general10)
            #x_vector = addDim(encoded_data_general)
            #x_vector = addDim(encoded_data_general_mean)
        elif i == 2:
            cols_news =['MarketsNews']
            x_vector = addDim(encoded_data_market10)
            #x_vector = addDim(encoded_data_market)
            #x_vector = addDim(encoded_data_market_mean)
        else:
            cols_news =['StockNews']
            x_vector = addDim(encoded_data_stock10)
            #x_vector = addDim(encoded_data_stock)
            #x_vector = addDim(encoded_data_stock_mean)


        
        x_vector = lookback(x_vector, n_past)

        stock_data = stock_data[(stock_data.index > start) & (stock_data.index < end)]
        sd_filtered = stock_data.filter(['Open', 'Close'])
        
        x_vector_train, x_prices_train,x_sentiments_train, y_train1, y_train2, x_vector_test, x_prices_test, x_sentiments_test, y_test1, y_test2, x_vector_valid, x_prices_valid, x_sentiments_valid, y_valid1, y_valid2 = splitTrainTest(x_vector, x_prices, x_sentiments, y1, y2, stock_data, test_start, valid_start)
        #x_vector_train, x_prices_train,x_sentiments_train, y_train1, y_train2, x_vector_test, x_prices_test, x_sentiments_test, y_test1, y_test2 = splitTrainTest_noValid(x_vector, x_prices, x_sentiments, y1, y2, stock_data, test_start, valid_start)

        y_train_org = y_train2.copy()
        y_valid_org = y_valid2.copy()
 
        
        # Modelling 
        #x_vector = None
        x_sentiments = None
        prices = False


        # Model ALL
        if not x_vector is None and not x_prices is None and not x_sentiments is None:
            if os.path.exists('weights.h5'):
                os.remove('weights.h5')
            
            pred_model = modelClass_All(n_past, news_amount)
            modelType = 'HybridOutput_incl_Embedding'
            history = pred_model.fit(x = [x_vector_train, x_sentiments_train, x_prices_train], 
                                y = [y_train1, y_train2], 
                                epochs = tr_epochs, 
                                callbacks=[es, rlr, mcp, tb], 
                                validation_data = ((x_vector_valid, x_sentiments_valid, x_prices_valid), [y_valid1, y_valid2]),
                                batch_size = batch_size)
          

            preds_train = pred_model.predict(x= [x_vector_train, x_sentiments_train, x_prices_train])
            preds_valid =  pred_model.predict(x= [x_vector_valid, x_sentiments_valid, x_prices_valid])
            preds_test = pred_model.predict(x= [x_vector_test, x_sentiments_test, x_prices_test])


            if re_train:
                data_test = stock_data[n_past + len(y_valid1) + len(y_train1):]
                datelist_test = data_test.index
                test_start = datelist_test[0].date()
                prev_day = str(test_start)
                rowCount = 0
                preds1 = None
                preds2 = None
                for i,d in enumerate(datelist_test):
                    if prev_day == str(d.date()):
                        rowCount = rowCount + 1
                    else:
                        test_start = str(d.date())
                        prev_day = str(d.date())
                        valid_start = str((pd.to_datetime(valid_start) + ONE_DAY).date())
                        new_preds = pred_model.predict(x = [x_vector_test[:rowCount],x_sentiments_test[:rowCount], x_prices_test[:rowCount]])
                        if preds1 is None:
                            preds1 = new_preds[0]
                            preds2 = new_preds[1]
                        else:
                            preds1 = np.concatenate((preds1, new_preds[0]))
                            preds2 = np.concatenate((preds2, new_preds[1]))


                        rowCount = 1
                        x_vector_train, x_prices_train,x_sentiments_train, y_train1, y_train2, x_vector_test, x_prices_test,x_sentiments_test, y_test1, y_test2, x_vector_valid, x_prices_valid, x_sentiments_valid, y_valid1, y_valid2 = splitTrainTest(x_vector, 
                                x_prices, x_sentiments, y1, y2, stock_data, test_start, valid_start)
                        pred_model.fit(x = [x_vector_train, x_sentiments_train, x_prices_train], y = [y_train1, y_train2], 
                                epochs = 2,  callbacks=[es, rlr, mcp, tb], validation_data = ((x_vector_valid, x_sentiments_valid, x_prices_valid), (y_valid1, y_valid2)), batch_size = 32)
                        #callbacks=[es, rlr, mcp, tb],

                new_preds = pred_model.predict(x = [x_vector_test[:rowCount], x_sentiments_test[:rowCount], x_prices_test[:rowCount]])
                preds1 = np.concatenate((preds1, new_preds[0]))
                preds2 = np.concatenate((preds2, new_preds[1]))

                preds_test_rt = [preds1, preds2]


        # Model incl. Sentiments
        elif prices and not x_sentiments is None:
            if os.path.exists('weights.h5'):
                os.remove('weights.h5')

            pred_model = modelClass_no_Embedding(n_past)
            modelType = 'HybridOutput_Sentiment_and_Price'
            history = pred_model.fit(x = [x_sentiments_train, x_prices_train],
                                y = [y_train1, y_train2] , epochs = tr_epochs, 
                                callbacks=[es, rlr, mcp, tb],
                                validation_data = ((x_sentiments_valid, x_prices_valid), [y_valid1, y_valid2]), batch_size = batch_size)
            
            if os.path.exists('weights.h5'):
                pred_model = load_model('weights.h5')

            preds_train = pred_model.predict(x= [x_sentiments_train, x_prices_train])
            preds_valid =  pred_model.predict(x= [x_sentiments_valid, x_prices_valid])
            preds_test = pred_model.predict(x= [x_sentiments_test, x_prices_test])


        #Model incl. Embeddings
        elif prices and not x_vector is None:
            pred_model = modelClass_Embedd_Price(n_past, news_amount)
            modelType = 'HybridOutput_Prices_Embeddings_' + emType + ' ' + feature
            history = pred_model.fit([x_prices_train, x_vector_train],   y = [y_train1,y_train2] ,epochs = tr_epochs, 
                                callbacks=[es, rlr, mcp, tb], 
                                validation_data = ((x_prices_valid, x_vector_valid), [y_valid1, y_valid2]), batch_size = batch_size)

            preds_train = pred_model.predict(x= [x_prices_train, x_vector_train])
            preds_valid =  pred_model.predict(x= [ x_prices_valid, x_vector_valid])
            preds_test = pred_model.predict(x= [ x_prices_test, x_vector_test])
        
        #Model only Vectors
        elif not x_vector is None and not prices and x_sentiments is None:
            if os.path.exists('weights.h5'):
                os.remove('weights.h5')

            pred_model = modelClass_onlyEmbedd(n_past)
            modelType = 'HybridOutput_just_Vectors'
            history = pred_model.fit(x_vector_train,  [y_train1, y_train2] ,epochs = tr_epochs, 
                callbacks=[es, rlr, mcp, tb], 
                validation_data = (x_vector_valid, [y_valid1,y_valid2]), batch_size = batch_size)
                        
            if os.path.exists('weights.h5'):
                pred_model = load_model('weights.h5')

            preds_train = pred_model.predict(x= x_vector_train)
            preds_valid =  pred_model.predict(x=  x_vector_valid)
            preds_test = pred_model.predict(x=  x_vector_test)


        #Model only Sentiments
        elif not x_sentiments is None and not prices is None and x_vector is None:
            if os.path.exists('weights.h5'):
                os.remove('weights.h5')

            pred_model = modelClass_onlySent(n_past)
            modelType = 'HybridOutput_just_Sentiments'
            history = pred_model.fit(x_sentiments_train,  [y_train1, y_train2] ,epochs = tr_epochs, 
                    callbacks=[es, rlr, mcp, tb], 
                    validation_data = (x_sentiments_valid, [y_valid1,y_valid2]), batch_size = batch_size)
                        
            if os.path.exists('weights.h5'):
                pred_model = load_model('weights.h5')

            preds_train = pred_model.predict(x= x_sentiments_train)
            preds_valid =  pred_model.predict(x=  x_sentiments_valid)
            preds_test = pred_model.predict(x=  x_sentiments_test)


        #Modell only Stock
        else:
            if os.path.exists('weights.h5'):
                os.remove('weights.h5')

            pred_model = modelClass_only_stock(n_past)
            modelType = 'HybridOutput_just_technical'
            history = pred_model.fit(x_prices_train,  [y_train1, y_train2] ,epochs = tr_epochs, 
                    callbacks=[es, rlr, mcp, tb], 
                    validation_data = (x_prices_valid, [y_valid1,y_valid2]), batch_size = batch_size)
            
            if os.path.exists('weights.h5'):
                pred_model = load_model('weights.h5')

            preds_train = pred_model.predict(x= x_prices_train)
            preds_valid =  pred_model.predict(x=  x_prices_valid)
            preds_test = pred_model.predict(x=  x_prices_test)
       

        train_loss = history.history['loss']
        val_loss = history.history['val_loss']

        # Plot the loss values
        plt.plot(train_loss, label='Training loss')
        plt.plot(val_loss, label='Validation loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        plt.show()

    
        if x_vector is None:
            cols_news = [""]
        cols_sent = cols_sentiment
        if x_sentiments is None:
            cols_sent = [""]
        if x_prices is None:
            cols_stock = [""]

        if run == 0 and i == 0:
            file_path =  generateOutputFile(modelType)
            
        print("#" * 30)
        print("Results Train Model:")
        RMSE_train, acc_train, act_return_train, acc_return_per_train = evalModel(preds_train, sd_filtered[n_past: n_past + len(y_train_org)], "Training", sc_predict)
        print("#" * 30)
        print("Results Validation Model:")
        RMSE_valid, acc_valid, act_return_valid, acc_return_per_valid = evalModel(preds_valid, sd_filtered[n_past + len(y_train_org): n_past + len(y_valid_org) + len(y_train_org)], "Validation", sc_predict)
        print("-" * 30)
        print("Results Test Model")
        RMSE_test, acc_test, act_return_test, acc_return_per_test = evalModel(preds_test, sd_filtered[n_past + len(y_train_org) + len(y_valid_org):], "Test", sc_predict)
        
        if re_train:
            print("Results Re Trained Test Model")
            RMSE_test_rt, acc_test_rt, act_return_test_rt, acc_return_per_test_rt = evalModel(preds_test_rt, sd_filtered[n_past  + len(y_train_org) + len(y_valid_org):], "Test", sc_predict)

        row_train= [ticker, cols_stock + cols_sent + cols_news, n_past, re_train, RMSE_train, acc_train, act_return_train, acc_return_per_train,  tr_epochs, batch_size, start, valid_start, model_type, 'Train']
        row_test = [ticker, cols_stock + cols_sent +cols_news, n_past, re_train, RMSE_test, acc_test, act_return_test, acc_return_per_test,  tr_epochs, batch_size,test_start, end, model_type, 'Test']
        if re_train:
            row_test_rt = [ticker, cols_stock + cols_sent + cols_news, n_past, re_train, RMSE_test_rt, acc_test_rt, act_return_test_rt, acc_return_per_test_rt,  tr_epochs, batch_size, test_start, end, model_type, 'Test Re Trained']
        
        with open(file_path, 'a') as res:
            writer_object = writer(res)
            writer_object.writerow(row_train)
            writer_object.writerow(row_test)
            if re_train:
                writer_object.writerow(row_test_rt)
            res.close()


In [ ]:
# Plot model and save to file
from IPython.display import SVG
from keras.utils import plot_model
from keras.utils.vis_utils import model_to_dot

plot_model(pred_model, to_file='data/thesis/modelStock.png')

#SVG(model_to_dot(model).create(prog='dot', format='svg'))

In [ ]:
df_train = df_complete[(df_complete.index > start) & (df_complete.index < valid_start)]
df_valid = df_complete[(df_complete.index >= valid_start) & (df_complete.index < test_start)]
df_test = df_complete[(df_complete.index >= test_start) & (df_complete.index < end)]

In [ ]:
df_complete.to_csv('datasetcomplete.csv')

In [ ]:
df_complete['GeneralNewsCounter'].sum()

In [ ]:
start, valid_start, test_start, end

In [ ]:
res = df_test.copy()

In [ ]:
res['MarketsNewsPosCounter'].sum(), res['MarketsNewsNegCounter'].sum(), res['MarketsNewsCounter'].sum()